## Reference Testing Packages, Scripts, and Notebooks using PyTest

1. **Make your tests discoverable by your test runner**

For Pytest, this usually means putting all test functions into files with filenames starting with `test_` (e.g. `test_utils.py`)

2. **Write test functions (e.g. `def test_fun1()`) in the test files**

For Modules:

```python
def test_detects_adult():
    from titanic_utils import is_adult
    assert is_adult(18) == True
```

For Scripts:

```python
def test_script_runs_without_errors():
    import runpy
    runpy.run_path('scripts/my_script.py')
```


For Jupyter Notebooks:

```python
def test_notebook_runs_without_errors():
    import subprocess
    subprocess.check_output("jupyter nbconvert --to notebook --execute my_notebook.ipynb".split())
```

### Create a GitHub Action that executes on Push

Example:

```
name: Run Tests

on: 
  push:
    branches:
      - main
      - dev
  pull_request:
    branches:
      - main
      - dev
  workflow_dispatch:

jobs:
  build-linux:
    runs-on: ubuntu-latest
    strategy:
      max-parallel: 5

  steps:
    - uses: actions/checkout@v3
    - name: Set up Python 3.8
      uses: actions/setup-python@v3
      with:
        python-version: 3.8
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install pytest
        pip install -e .
    - name: Test with pytest
      run: pytest
```

## Reference: Publishing Python Packages on PyPI using Twine and GitHub Actions

Quick Command Reference:

| File or Command | Description |
| :-- | :-- | 
| `pyproject.toml`| Put in at least `[build-system]` and `[project]` sections |
| `pip install build; python -m build` | Builds files in `dist/` | 
| `twine upload dist/*` | Upload `dist/`|


 ### Make a `pyproject.toml` file in the Project Root directory


1. **Select a build system (here, uses setuptools):**

```
[build-system]
requires = ["setuptools>=61.0"]
build-backend = "setuptools.build_meta"
```

2. **Configure the [Project's Metadata](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#declaring-project-metadata)** (most entries are optional, all are helpful):

```
[project]
name = "spam"                            
version = "0.1.0"
description = "Lovely Spam! Wonderful Spam!"
readme = "README.rst"
requires-python = ">=3.8"
license = {file = "LICENSE.txt"}
keywords = ["egg", "bacon", "sausage", "tomatoes", "Lobster Thermidor"]
authors = [
  {email = "pradyun@example.com"},
  {name = "Tzu-Ping Chung"}
]
maintainers = [
  {name = "Brett Cannon", email = "brett@python.org"}
]

# https://martin-thoma.com/software-development-stages/ 
classifiers = [
  "Programming Language :: Python",
  "Development Status :: 4 - Beta" 
]

dependencies = [
  "httpx",
  "gidgethub[httpx]>4.0.0",
  "django>2.1; os_name != 'nt'",
  "django>2.0; os_name == 'nt'"
]
# dynamic = ["version", "description"]

# If using requirements.txt file:
# dynamic = ["dependencies"]
# [tool.setuptools.dynamic]
# dependencies = {file = ["requirements.txt"]}

# If using setuptools_scm for versioning using git tags:
# dynamic = ["version"]
# [tool.setuptools_scm]
# write_to = "pkg/_version.py"
# no-local-version = true

[project.optional-dependencies]
test = [
  "pytest > 5.0.0",
  "pytest-cov[all]"
]
doc = [
  "sphinx",
  "furo"
]

[project.urls]
homepage = "https://example.com"
documentation = "https://readthedocs.org"
repository = "https://github.com/me/spam.git"
changelog = "https://github.com/me/spam/blob/master/CHANGELOG.md"

[project.scripts]
spam-cli = "spam:main_cli"

[project.gui-scripts]
spam-gui = "spam:main_gui"

[project.entry-points."spam.magical"]
tomatoes = "spam:main_tomatoes"
```

### Build the Distribution Files

3. **Update the Version Number**

Every version number for a release must be unique.  If using setuptools-scm, this will be taken care of automatically, but you may still want to make a new version number using a git tag.

```
git tag -a v1.4 -m "my version 1.4"  # Should match the version number elsewhere
```

4. **Build the Distribution files in `dist/`**

```
pip install build
python -m build
```

5. **Verify that the distribution is correct and complete**

Unzip the `.whl` file and check that the files you want are there.

### Upload the Files

#### First time only: check that the uploads are working properly

6. **Upload to Test PyPI: Check that everything's ready for Upload**

```
pip install twine
twine check dist/*
```

7. **Upload to Test PyPI: Register a Username to the [Test PyPI Repository](https://test.pypi.org/)**

Go to the test.pypi.org website in your web browser, check your login works

8. **Upload to Test PyPI ([Reference](https://packaging.python.org/en/latest/tutorials/packaging-projects/#uploading-the-distribution-archives))**

Note that the username and password can be put in interactively, via the command line (`-u` and `-p` flags), or using environment variables (`TWINE_USERNAME` and `TWINE_PASSWORD`).  Also, tokens can be used instead of user passwords, gotten via the PyPI website.

```
twine upload --repository-url=https://test.pypi.org/legacy/ dist/*
```

9. **Verify the upload is correct.**

Visit the PyPI page for the release, Download the files, and unzip them.  Also check that your readme and metadata were rendered properly in the page.

### Every time: Upload the New Release

10. **Upload to Real PyPI: Register a Username to https://pypi.org/**

Go to the https://pypi.org/ website in your web browser, check your login works

11. **Upload to Real PyPI ([Reference](https://packaging.python.org/en/latest/tutorials/packaging-projects/#uploading-the-distribution-archives))**

Note that the username and password can be put in interactively, via the command line (`-u` and `-p` flags), or using environment variables (`TWINE_USERNAME` and `TWINE_PASSWORD`).  Also, tokens can be used instead of user passwords, gotten via the PyPI website.

```
twine upload dist/*
```

12. **Verify the upload is correct.**

Visit the PyPI page for the release, Download the files, and unzip them.  Also check that your readme and metadata were rendered properly in the page.

## Github Actions: Deploy on Tagged Push or Release


### Create a GitHub Actions Workflow

1. **Add Secrets to the GitHUB Repository Secrets**

Tip: Best to use a specific PyPI API Token for this, instead of your username and password

2. **Add the Release Step to a GitHub Actions Workflow**

Useful Triggers:

```
on: 
  push:
    branches:
      - main
    tags:
      - "v*" # Push events to matching v*, i.e. v1.0, v20.15.10
  release:
      types:
        - published
  workflow_dispatch:
```

Add a new job (either in an existing workflow, or in a new workflow)

```
jobs: 
   deploy:  # id of the job
      needs: [test]  # if the "test" job succeeded (in same workflow), then this job can run.
      runs-on: ubuntu-latest
      if: contains(github.ref, 'tags')   
```

Option A: Create a New Workflow (Example below)

Specify the release steps in the job (Example below):

```
steps:
  - uses: actions/checkout@v2
  - name: Set up Python
    uses: actions/setup-python@v2
    with:
      python-version: "3.x"
  - name: Install dependencies
    run: |
      python -m pip install --upgrade pip
      pip install -U setuptools setuptools_scm wheel twine build
  - name: Build and publish
    env:
      TWINE_USERNAME: __token__
      TWINE_PASSWORD: ${{ secrets.TWINE_API_KEY }}
    run: |
      git tag
      python -m build .
      twine upload dist/*
```

8. **Make a New Release**

Click the "Create a new release" button in the right side of your Github repo homepage, make a new tag, and hit the publish button.  Once published, you should see the GitHub Action start and if successful, a new version should be on PyPI!